# Residual Networks (ResNet)

In [1]:
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import data as gdata, nn
from mxnet.gluon import loss as gloss
import mxnet as mx
import os
import sys
import time

In [2]:
class Residual(nn.Block):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.conv1 = nn.Conv2D(num_channels, kernel_size=3, padding=1, strides=strides)
        self.conv2 = nn.Conv2D(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2D(num_channels, kernel_size=1, strides=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm()
        self.bn2 = nn.BatchNorm()

    def forward(self, X):
        Y = nd.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return nd.relu(Y + X)

In [3]:
blk = Residual(3)
blk.initialize()
X = nd.random.uniform(shape=(4, 3, 6, 6))
blk(X).shape

(4, 3, 6, 6)

In [4]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk.initialize()
blk(X).shape

(4, 6, 3, 3)

### ResNet Model Stage 1

ResNet and GoogLeNet are quite similar on the initial layers.

In [5]:
net = nn.Sequential()
net.add(nn.Conv2D(64, kernel_size=7, strides=2, padding=3),
        nn.BatchNorm(), nn.Activation('relu'),
        nn.MaxPool2D(pool_size=3, strides=2, padding=1))

We also need a ResNet block.

In [6]:
def resnet_block(num_channels, num_residuals, first_block=False):
    blk = nn.Sequential()
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.add(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            blk.add(Residual(num_channels))
    return blk

Then, we add all the residual blocks to ResNet. Here, two residual blocks are used for each module.

In [7]:
net.add(resnet_block(64, 2, first_block=True),
        resnet_block(128, 2),
        resnet_block(256, 2),
        resnet_block(512, 2))

Finally, just like GoogLeNet, we add a global average pooling layer, followed by the fully connected layer output.

In [8]:
net.add(nn.GlobalAvgPool2D(), nn.Dense(38))

### Full ResNet-18


In [9]:
X = nd.random.uniform(shape=(1, 1, 224, 224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

conv5 output shape:	 (1, 64, 112, 112)
batchnorm4 output shape:	 (1, 64, 112, 112)
relu0 output shape:	 (1, 64, 112, 112)
pool0 output shape:	 (1, 64, 56, 56)
sequential1 output shape:	 (1, 64, 56, 56)
sequential2 output shape:	 (1, 128, 28, 28)
sequential3 output shape:	 (1, 256, 14, 14)
sequential4 output shape:	 (1, 512, 7, 7)
pool1 output shape:	 (1, 512, 1, 1)
dense0 output shape:	 (1, 38)


In [16]:
def get_dataloader_workers(num_workers=4):
    # 0 means no additional process is used to speed up the reading of data.
    if sys.platform.startswith('win'):
        return 0
    else:
        return num_workers

def load_data_fashion_mnist(batch_size, resize=None):
    data = gluon.data.vision
    trans = [data.transforms.Resize(resize)] if resize else []
    trans.append(data.transforms.ToTensor())
    trans = data.transforms.Compose(trans)
    mnist_train = gluon.data.vision.ImageFolderDataset("/kaggle/input/new-plant-diseases-dataset/train", flag=38).transform_first(trans)
    mnist_test = gluon.data.vision.ImageFolderDataset("/kaggle/input/new-plant-diseases-dataset/st", flag=38).transform_first(trans)
    return (gluon.data.DataLoader(mnist_train, batch_size, shuffle=True,
                                  num_workers=get_dataloader_workers()),
            gluon.data.DataLoader(mnist_test, batch_size, shuffle=False,
                                  num_workers=get_dataloader_workers()))

batch_size = 128
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

/opt/conda/lib/python3.7/site-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../input/freshrotten/FreshRotten/test/freshapples of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/opt/conda/lib/python3.7/site-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../input/freshrotten/FreshRotten/test/freshbanana of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/opt/conda/lib/python3.7/site-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../input/freshrotten/FreshRotten/test/freshoranges of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/opt/conda/lib/python3.7/site-packages/mxnet/gluon/data/vision/datasets.py:318: UserWarning: Ignoring ../input/freshrotten/FreshRotten/test/rottenapples of type . Only support .jpg, .jpeg, .png
  filename, ext, ', '.join(self._exts)))
/opt/conda/lib/python3.7/site-packages/mxnet/gluon/data/vision

In [17]:
def evaluate_accuracy(data_iter, net, ctx):
    acc_sum, n = nd.array([0], ctx=ctx), 0
    for X, y in data_iter:
        # If ctx is the GPU, copy the data to the GPU.
        X, y = X.as_in_context(ctx), y.as_in_context(ctx).astype('float32')
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.size
    return acc_sum.asscalar() / n

def train(net, train_iter, test_iter, batch_size, trainer, ctx,
              num_epochs):
    print('training on', ctx)
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc = evaluate_accuracy(test_iter, net, ctx)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [2]:
lr, num_epochs, batch_size, ctx = 0.05, 5, 256, mx.gpu()
net.initialize(force_reinit=True, ctx=ctx, init=init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)
train(net, train_iter, test_iter, batch_size, trainer, ctx, num_epochs)


training on gpu(0)
Epoch 1/5
13600/13600 [==============================] - 6s 463us/step - loss: 0.0413 - accuracy: 0.9800 - val_loss: 0.0336 - val_accuracy: 0.9724
Epoch 2/5
13600/13600 [==============================] - 7s 481us/step - loss: 0.0381 - accuracy: 0.9832 - val_loss: 0.0312 - val_accuracy: 0.9750
Epoch 3/5
13600/13600 [==============================] - 6s 467us/step - loss: 0.0365 - accuracy: 0.9847 - val_loss: 0.0260 - val_accuracy: 0.9815
Epoch 4/5
13600/13600 [==============================] - 6s 466us/step - loss: 0.0349 - accuracy: 0.9827 - val_loss: 0.0333 - val_accuracy: 0.9768
Epoch 5/5
13600/13600 [==============================] - 6s 473us/step - loss: 0.0336 - accuracy: 0.9843 - val_loss: 0.0218 - val_accuracy: 0.9829
saved
